#Created by WMS, modified by MEK

The UCLA miniscope system produces unqiue timestamp logs for each miniscope and behavioral camera used in a session. This script:

1) loads timestamps for miniscope, minicam (single or multiple), and cleversys behavioral event output 

IMPORTANT: this script assumes that data directory is organized as .../session_date/trial_time/devices

### Step 0: load timestamp logs (either multiple or one per device per session)

In [11]:
import pandas as pd
import os
import re
import sys
#!{sys.executable} -m pip install openpyxl
import openpyxl #for loading cleversys xlsx
import pdb #for debugging
from matplotlib import pyplot as plt


#!{sys.executable} -m pip install xlrd
#import xlrd


#pip install yapf
#from yapf.yapflib.yafp_api import FormatCode

#identify imaging session directory
#sess_dir = "C://miniscopeData/5546/2023_01_01/13_17_27/01012023_PPTnodrug/"


sess_dir = "C://miniscopeData/5546/2022_12_15/13_56_20_record/12152022_cohabitation_5546/"
sess_dir = "C://miniscopeData/5546/2022_12_16/14_36_12/121622_cohousing_5546/"
sess_dir = "C://miniscopeData/5546/2023_01_01/13_17_27/01012023_PPTnodrug/"
sess_dir = "C://miniscopeData/5546/2022_12_17/12_48_22/"



writeOutFolder = os.path.join(sess_dir + 'alignerOutput/')
writeCleversysCsvOutPath = os.path.join(writeOutFolder + 'cleversysevents.csv')
writeBorisCsvOutPath = os.path.join(writeOutFolder + 'borisevents.csv')

if not os.path.exists(writeOutFolder):
    os.makedirs(writeOutFolder)
frameRate = 50


#a = os.listdir(sess_dir)

#put all folders with HH_MM_SS format into list
#trials = []

#date_patt = "[0-2][0-9]_[0-5][0-9]_[0-5][0-9]"

#for i in a:
#    if re.search(date_patt, i):
#        trials.append(i)

#place in chronological order
#trials.sort()

In [12]:
#for loop that goes into each folder in trials, opens the MiniCam1, MiniCam2, and My_V4_Miniscope folders 
#and grabs the timestamp .csv from each

ms1 = "My_V4_Miniscope" #may need to change this with multi-scope setup
#ms2 = "My_V4_Miniscope2" #i don't actually yet know what the nomenclature will be for a second scope
mc1 = "MiniCam2"
mc2 = "MiniCam2"
cv = "cleversys"

devices = [ms1, mc1, cv] #ms2

#currently written assuming only cam 1 is the top-down

ts_files = []


ms1_ts_files = []
#ms2_ts_files = []
mc1_ts_files = []
mc2_ts_files = []
cv_event_files = []

#can nest these for loops likely
#for j in trials:
ms1_ts_path = os.path.join(sess_dir, ms1, "timeStamps.csv").replace("\\","/")
ms1_ts_files.append(ms1_ts_path)
    
#for k in trials:
#    ms2_ts_path = os.path.join(sess_dir, k, ms2, "timeStamps.csv").replace("\\","/")
#    ms2_ts_files.append(ms2_ts_path)

#for l in trials:
mc1_ts_path = os.path.join(sess_dir, mc1, "timeStamps.csv").replace("\\","/")
mc1_ts_files.append(mc1_ts_path)
    
#for m in trials:
#mc2_ts_path = os.path.join(sess_dir, j, mc2, "timeStamps.csv").replace("\\","/")
#mc2_ts_files.append(mc2_ts_path)

cv_ts_path = os.path.join(sess_dir, cv, "events1.xlsx").replace("\\","/")
cv_event_files.append(cv_ts_path)

### Step 1: concatenating each device's timestamp logs (not necessary if you only have one recording per device per session)

In [3]:
cv_event_files

['C://miniscopeData/5546/2022_12_16/14_36_12/12162022_cohousing_5546/cleversys/events1.xlsx']

I assume the videoWriter function in the video concat script immediately places one videos' frames after the previous (ie frame 0 on second video becomes frame n+1 when n = last frame of first video). We should confirm this.

I also assume that Bento or other things importing concatenated files will want timestamps to be continuous integers throughout all the timestamps, not something like 1.123345, 2.12043034, where the prefix would denote which original file the timestamp came from

With those assumptions, the strategy in this step is to:

a) load first timestamp file from all streams (assuming your trials are ordered correctly, see above cell)

b) find maximum  frame # of each timestamp file (ms1_max = x, mc1_max = y, mc2_max = z) and save each as its own variable

c) find maximum timestamp of each timestamp file and compare to find the latest timestamp across the three streams' first video

d) save this max timestamp as its own variable and add 60000 (as a one minute spacer for a faux start time for the next video)

e) save each timestamp file into a new table

f) load next timestamp file for all streams

g) repeat b)

h) add the max frame from the previous file to each of the current frame numbers

i) repeat c) and d)

j) add the value calculated in d) to all timestamps in all loaded files

k) concatenate each timestamp file to previous file

l) repeat f-k for remaining files 

In [4]:
cv

'cleversys'

In [13]:
#IF RUNNING BORIS ONLY
#read ts files into dfs - any reason to make these a dictionary?
mscope1 = pd.read_csv(ms1_ts_files[0])
#mscope2 = pd.read_csv(ms2_ts_files[p])
mcam1 = pd.read_csv(mc1_ts_files[0])

In [14]:
#for p in range(len(trials)):
#read ts files into dfs - any reason to make these a dictionary?
mscope1 = pd.read_csv(ms1_ts_files[0])
#mscope2 = pd.read_csv(ms2_ts_files[p])
mcam1 = pd.read_csv(mc1_ts_files[0])
#mcam2 = pd.read_csv(mc2_ts_files[p])
cv = pd.read_excel(cv_event_files[0])

cv.rename(columns=cv.iloc[0]).drop(cv.index[0])
    
# if p == 0:
        #find max frame of each
ms1_maxF = mscope1['Frame Number'].max()
mc1_maxF = mcam1['Frame Number'].max()
#mc2_maxF = mcam2['Frame Number'].max()
    
#find max timestamp across all files, add 60000
ms1_maxTS = mscope1['Time Stamp (ms)'].max()
mc1_maxTS = mcam1['Time Stamp (ms)'].max()
#mc2_maxTS = mcam2['Time Stamp (ms)'].max()
#maxTSs = [ms1_maxTS, mc1_maxTS, mc2_maxTS]
maxTSs = [ms1_maxTS, mc1_maxTS]
        
#next_start_time = (max(maxTSs) + 60000)

#compile dfs into overall dfs
#ms1_concat = mscope1
#ms2_concat = mscope2
#mc1_concat = mcam1
#mc2_concat = mcam2


# Rename columns, add columns for frame start/stop
cv.columns= ['EventID','StartinSec','EndinSec','LengthinSec','EventType']
startTimes = cv['StartinSec']
startTimesInFrames = (startTimes * frameRate)
cv['startInFrames'] = startTimesInFrames

#pdb.set_trace()

endTimes = cv['EndinSec']
endTimesInFrames = (endTimes * frameRate)
cv['endInFrames'] = endTimesInFrames


cv = cv.tail(-1) #remove first header row

cv.startInFrames = cv.startInFrames.astype(int) #otherwise a combo of ints & floats for some reason
cv.endInFrames = cv.endInFrames.astype(int)
cv = cv.round({'startInFrames': 0}) #round to nearest integer
cv = cv.round({'endInFrames': 0})
cv
    
        

,EventID,StartinSec,EndinSec,LengthinSec,EventType,startInFrames,endInFrames
1,1,6,12.32,6.32,Area:Mouse 1 Center In Center,300,616
2,2,6,7.9,1.9,Area:Mouse 1 Center In Center chamber,300,395
3,3,7.9,43.04,35.14,Area:Mouse 1 Center In Right chamber,395,2152
4,4,7.9,12.32,4.42,Area:Mouse 1 Center In Taunt right,395,616
5,5,9.52,10.88,1.36,Mouse 1 Locomotion in Area Center,476,544
...,...,...,...,...,...,...,...
2092,2092,3394.12,3394.14,0.02,Mouse 1 Locomotion in Area Center chamber,169706,169707
2093,2093,3394.12,3394.14,0.02,Mouse 1 Locomotion in Area Taunt left,169706,169707
2094,2094,3394.12,3394.14,0.02,Mouse 1 Locomotion in Area Taunt right,169706,169707
2095,2095,3400.3,3410.96,10.66,Social Contact [ 1 with 2 ] in Area Left while...,170015,170548


In [18]:
temp = cv[cv['EventType'] == 'Social Contact [ 1 with 3 ] in Area Right while Joint Motion < 0.030']
print(sum(temp.LengthinSec))

temp = cv[cv['EventType'] == 'Social Contact [ 1 with 2 ] in Area Left while Joint Motion < 0.030']
print(sum(temp.LengthinSec))

63.96000000000001
759.0600000000001


In [6]:
#CLEVERSYS EVENT FILE TIMESTAMPS TO MINISCOPE FRAME INDEX

#for each behavior start and end time in frames, go into minicam and pull the corresponding timestamps
#using that timestamp, find nearest miniscope row
#write that info back into the cleversys dataframe and save it

#NOTE-the scopeFrame indices do not necessarily index Minian output appropriately
#e.g if there has been temporal downsampling


starts = cv.startInFrames
starts = starts.to_dict()
ends = cv.endInFrames
ends  = ends.to_dict()

timestampStart = {}
timestampEnd = {}
scopeFrameStart = {}
scopeFrameEnd = {}



for l in starts:
    indx = starts[l]
    #mc1_concat.iloc[[starts],1]
    timestampStart[l] = mcam1.iloc[indx,1] #grab the timestamp of that minicam frame
    indxEnd = ends[l]
    try: #to account for trying to grab one frame beyond what is in minicam
        timestampEnd[l] = mcam1.iloc[indxEnd,1] #grab the timestamp of that minicam frame
    except:
        timestampEnd[l] = mcam1.iloc[indxEnd-1,1]
    closest = mscope1.iloc[(mscope1['Time Stamp (ms)']-timestampStart[l]).abs().argsort()[:1]]
    scopeFrameStart[l] = round(closest.iloc[0,0] / 2)
    closest = mscope1.iloc[(mscope1['Time Stamp (ms)']-timestampEnd[l]).abs().argsort()[:1]]
    scopeFrameEnd[l] = round(closest.iloc[0,0] / 2)
    
    
#write back into cleversys dataframe
cv['startInMs'] = timestampStart.values()
cv['endInMs'] = timestampEnd.values()
cv['scopeFrameStart'] = scopeFrameStart.values() #frame # of the row with the nearest timestamp
cv['scopeFrameEnd'] = scopeFrameEnd.values()


# save updated dataframe out 
cv.to_csv(writeCleversysCsvOutPath)

cv




,EventID,StartinSec,EndinSec,LengthinSec,EventType,startInFrames,endInFrames,startInMs,endInMs,scopeFrameStart,scopeFrameEnd
1,1,6,12.32,6.32,Area:Mouse 1 Center In Center,300,616,6378,13116,96,196
2,2,6,7.9,1.9,Area:Mouse 1 Center In Center chamber,300,395,6378,8406,96,126
3,3,7.9,43.04,35.14,Area:Mouse 1 Center In Right chamber,395,2152,8406,45840,126,683
4,4,7.9,12.32,4.42,Area:Mouse 1 Center In Taunt right,395,616,8406,13116,126,196
5,5,9.52,10.88,1.36,Mouse 1 Locomotion in Area Center,476,544,10129,11577,152,173
...,...,...,...,...,...,...,...,...,...,...,...
2092,2092,3394.12,3394.14,0.02,Mouse 1 Locomotion in Area Center chamber,169706,169707,3620467,3620492,53914,53914
2093,2093,3394.12,3394.14,0.02,Mouse 1 Locomotion in Area Taunt left,169706,169707,3620467,3620492,53914,53914
2094,2094,3394.12,3394.14,0.02,Mouse 1 Locomotion in Area Taunt right,169706,169707,3620467,3620492,53914,53914
2095,2095,3400.3,3410.96,10.66,Social Contact [ 1 with 2 ] in Area Left while...,170015,170548,3627051,3638410,54012,54181


In [16]:
scopeFrameStart

NameError: name 'scopeFrameStart' is not defined

In [26]:
# save updated dataframe out 
#cv.to_csv(writeCsvOutPath)
#type(boris1)

In [9]:
# BORIS EVENT TIMESTAMPS TO MINISCOPE FRAME INDEX
# same general procedure as last cell
borisInput = os.path.join(sess_dir + 'minicam2_compiled/output/aggregated_run1.tsv/')

boris1 = pd.read_csv(borisInput,sep='\t')


# get starting seconds, multiply by FPS to get frames, then round to integer
borisStarts = boris1['Start (s)']
borisStartsInFrames = (borisStarts * frameRate)
boris1['startFrameNum'] = borisStartsInFrames
boris1['startFrameNum'] = boris1['startFrameNum'].round(0).astype(int)

borisEnds = boris1['Stop (s)']
borisEndsInFrames = (borisEnds * frameRate)
boris1['endFrameNum'] = borisEndsInFrames
boris1['endFrameNum'] = boris1['endFrameNum'].round(0).astype(int)


boris1


starts = boris1.startFrameNum
starts = starts.to_dict()
ends = boris1.endFrameNum
ends  = ends.to_dict()

timestampStart = {}
timestampEnd = {}
scopeFrameStart = {}
scopeFrameEnd = {}



for l in starts:
    indx = starts[l]
    #mc1_concat.iloc[[starts],1]
    timestampStart[l] = mcam1.iloc[indx,1] #grab the timestamp of that minicam frame
    indxEnd = ends[l]
    try: #to account for trying to grab one frame beyond what is in minicam
        timestampEnd[l] = mcam1.iloc[indxEnd,1] #grab the timestamp of that minicam frame
    except:
        timestampEnd[l] = mcam1.iloc[indxEnd-1,1]
    closest = mscope1.iloc[(mscope1['Time Stamp (ms)']-timestampStart[l]).abs().argsort()[:1]]
    scopeFrameStart[l] = round(closest.iloc[0,0] / 2)
    closest = mscope1.iloc[(mscope1['Time Stamp (ms)']-timestampEnd[l]).abs().argsort()[:1]]
    scopeFrameEnd[l] = round(closest.iloc[0,0] / 2)
    
    
#write back into cleversys dataframe
#boris1['startInMs'] = timestampStart.values()
#boris1['endInMs'] = timestampEnd.values()
boris1['scopeFrameStart'] = scopeFrameStart.values() #frame # of the row with the nearest timestamp
boris1['scopeFrameEnd'] = scopeFrameEnd.values()


# save updated dataframe out 
boris1.to_csv(writeBorisCsvOutPath)

boris1




,Observation id,Observation date,Description,Media file,Total length,FPS,Subject,Behavior,Behavioral category,Modifiers,Behavior type,Start (s),Stop (s),Duration (s),Comment start,Comment stop,startFrameNum,endFrameNum,scopeFrameStart,scopeFrameEnd
0,5546_0101_1317,2023-01-22 16:59:48,NaN,C:/miniscopeData/5546/2023_01_01/13_17_27/0101...,3543.051,50.0,Left,noncontact investigation,Investigation,Initiating,STATE,12.860,17.600,4.740,NaN,NaN,643,880,204,279
1,5546_0101_1317,2023-01-22 16:59:48,NaN,C:/miniscopeData/5546/2023_01_01/13_17_27/0101...,3543.051,50.0,Left,noncontact investigation,Investigation,Initiating,STATE,22.940,24.240,1.300,NaN,NaN,1147,1212,364,384
2,5546_0101_1317,2023-01-22 16:59:48,NaN,C:/miniscopeData/5546/2023_01_01/13_17_27/0101...,3543.051,50.0,Left,noncontact investigation,Investigation,Initiating,STATE,26.400,28.500,2.100,NaN,NaN,1320,1425,418,452
3,5546_0101_1317,2023-01-22 16:59:48,NaN,C:/miniscopeData/5546/2023_01_01/13_17_27/0101...,3543.051,50.0,Left,contact investigation,Affiliative,Initiating,STATE,28.540,30.600,2.060,NaN,NaN,1427,1530,452,486
4,5546_0101_1317,2023-01-22 16:59:48,NaN,C:/miniscopeData/5546/2023_01_01/13_17_27/0101...,3543.051,50.0,Left,noncontact investigation,Investigation,Initiating,STATE,37.680,41.700,4.020,NaN,NaN,1884,2085,598,662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,5546_0101_1317,2023-01-22 16:59:48,NaN,C:/miniscopeData/5546/2023_01_01/13_17_27/0101...,3543.051,50.0,Left,food (eating,NaN,NaN,STATE,3473.880,3476.460,2.580,NaN,NaN,173694,173823,55124,55166
351,5546_0101_1317,2023-01-22 16:59:48,NaN,C:/miniscopeData/5546/2023_01_01/13_17_27/0101...,3543.051,50.0,Left,contact investigation,Affiliative,Initiating,STATE,3478.360,3484.240,5.880,NaN,NaN,173918,174212,55196,55289
352,5546_0101_1317,2023-01-22 16:59:48,NaN,C:/miniscopeData/5546/2023_01_01/13_17_27/0101...,3543.051,50.0,Left,grooming,Affiliative,Grooming self,STATE,3497.020,3506.460,9.440,NaN,NaN,174851,175323,55492,55642
353,5546_0101_1317,2023-01-22 16:59:48,NaN,C:/miniscopeData/5546/2023_01_01/13_17_27/0101...,3543.051,50.0,Left,grooming,Affiliative,Grooming self,STATE,3531.800,3534.951,3.151,NaN,NaN,176590,176748,56044,56094


In [10]:
import pickle 


with open(os.path.join(writeOutFolder + 'day14_forCaiman.pkl'), 'wb') as f:
    pickle.dump([cv, boris1], f)

In [7]:
# finally, load in Minian output and save out cleversys, boris, and minian files into a pkl


#get ready to load in Minian output
import holoviews as hv
import numpy as np
import xarray as xr

minian_path = "."
minian_ds_path = os.path.join(sess_dir, "My_V4_Miniscope\minian_output")

sys.path.append(minian_path)

from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)



In [8]:
import pickle 

#load Minian output, convert it to df, drop rows with NaN
minian_ds = open_minian(minian_ds_path)
C_dropped = minian_ds['C_dropped']
C_dropped_df = C_dropped.to_pandas()
cellsKept = C_dropped_df.dropna()
#cellsKept

S_dropped = minian_ds['S_dropped']
S_dropped_df = S_dropped.to_pandas()
cellsKeptSpikes = S_dropped_df.dropna()
#cellsKeptSpikes

#implement cross-correlation here to do further merging

with open(os.path.join(writeOutFolder + 'minianOut.pkl'), 'wb') as f:
    pickle.dump([cellsKept, cellsKeptSpikes, cv, boris1], f)


#cellsKept = cellsKeptSpikes


